In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline        


def process_image(videoframe):
    
    
    mtx = np.array([
                    [1.15662906e+03, 0.00000000e+00, 6.69041437e+02],
                    [0.00000000e+00, 1.15169194e+03, 3.88137239e+02],
                    [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])
    dist = np.array([[-0.2315715,  -0.12000538, -0.00118338,  0.00023305,  0.15641572]])


    def color_and_gradient(img, thresh_min=20, thresh_max=100, s_thresh_min=170, s_thresh_max=255):

        # Convert to HLS color space and separate the S channel
        # Note: img is the undistorted image
        hls = cv2.cvtColor(undist, cv2.COLOR_RGB2HLS)
        s_channel = hls[:,:,2]

        # Grayscale image
        # NOTE: we already saw that standard grayscaling lost color information for the lane lines
        # Explore gradients in other colors spaces / color channels to see what might work better
        gray = cv2.cvtColor(undist, cv2.COLOR_RGB2GRAY)

        # Sobel x
        sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0) # Take the derivative in x
        abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
        scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))

        # Threshold x gradient
        #thresh_min = 20
        #thresh_max = 100
        sxbinary = np.zeros_like(scaled_sobel)
        sxbinary[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1

        # Threshold color channel
        # s_thresh_min = 180
        # s_thresh_max = 255
        s_binary = np.zeros_like(s_channel)
        s_binary[(s_channel >= s_thresh_min) & (s_channel <= s_thresh_max)] = 1
        
        # Stack each channel to view their individual contributions in green and blue respectively
        # This returns a stack of the two binary images, whose components you can see as different colors
        color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) * 255

        # Combine the two binary thresholds
        combined_binary = np.zeros_like(sxbinary)
        combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1

        return combined_binary


    def unwarp(img):

        # Defininig 4 source points dst = np.float32([[,],[,],[,],[,]])
        src = np.float32([[200,720],     # left bottom
              [574,460],                 # left top
              [715,460],                 # right top
              [1120,712]])               # right bottom

        # Defininig 4 destination points dst = np.float32([[,],[,],[,],[,]])

        margin = 320

        dst = np.float32([[margin,img.shape[0]],      # left bottom
             [margin,0],                              # left top
             [img.shape[1]-margin,0],                 # right top
             [img.shape[1]-margin,img.shape[0]]])     # right bottom


        # d) use cv2.getPerspectiveTransform() to get M, the transform matrix
        M = cv2.getPerspectiveTransform(src, dst)
        Minv = cv2.getPerspectiveTransform(dst, src)
        # e) use cv2.warpPerspective() to warp your image to a top-down view
        warped = cv2.warpPerspective(binary, M, (binary.shape[1],binary.shape[0]), flags=cv2.INTER_LINEAR)

        return warped, Minv


    def find_lane_pixels(binary_warped):
        # Take a histogram of the bottom half of the image
        histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
        # Create an output image to draw on and visualize the result
        out_img = np.dstack((binary_warped, binary_warped, binary_warped))
        # Find the peak of the left and right halves of the histogram
        # These will be the starting point for the left and right lines
        midpoint = np.int(histogram.shape[0]//2)
        leftx_base = np.argmax(histogram[:midpoint])
        rightx_base = np.argmax(histogram[midpoint:]) + midpoint

        # HYPERPARAMETERS
        # Choose the number of sliding windows
        nwindows = 9
        # Set the width of the windows +/- margin
        margin = 100
        # Set minimum number of pixels found to recenter window
        minpix = 50

        # Set height of windows - based on nwindows above and image shape
        window_height = np.int(binary_warped.shape[0]//nwindows)
        # Identify the x and y positions of all nonzero pixels in the image
        nonzero = binary_warped.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Current positions to be updated later for each window in nwindows
        leftx_current = leftx_base
        rightx_current = rightx_base

        # Create empty lists to receive left and right lane pixel indices
        left_lane_inds = []
        right_lane_inds = []

        # Step through the windows one by one
        for window in range(nwindows):
            # Identify window boundaries in x and y (and right and left)
            win_y_low = binary_warped.shape[0] - (window+1)*window_height
            win_y_high = binary_warped.shape[0] - window*window_height
            win_xleft_low = leftx_current - margin
            win_xleft_high = leftx_current + margin
            win_xright_low = rightx_current - margin
            win_xright_high = rightx_current + margin

            # Draw the windows on the visualization image
            cv2.rectangle(out_img,(win_xleft_low,win_y_low),
            (win_xleft_high,win_y_high),(0,255,0), 2) 
            cv2.rectangle(out_img,(win_xright_low,win_y_low),
            (win_xright_high,win_y_high),(0,255,0), 2) 

            # Identify the nonzero pixels in x and y within the window #
            good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
            (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
            good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
            (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]

            # Append these indices to the lists
            left_lane_inds.append(good_left_inds)
            right_lane_inds.append(good_right_inds)

            # If you found > minpix pixels, recenter next window on their mean position
            if len(good_left_inds) > minpix:
                leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
            if len(good_right_inds) > minpix:        
                rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

        # Concatenate the arrays of indices (previously was a list of lists of pixels)
        try:
            left_lane_inds = np.concatenate(left_lane_inds)
            right_lane_inds = np.concatenate(right_lane_inds)
        except ValueError:
            # Avoids an error if the above is not implemented fully
            pass

        # Extract left and right line pixel positions
        leftx = nonzerox[left_lane_inds]
        lefty = nonzeroy[left_lane_inds] 
        rightx = nonzerox[right_lane_inds]
        righty = nonzeroy[right_lane_inds]
    
        return leftx, lefty, rightx, righty, out_img
        

    def fit_polynomial(binary_warped):
        
        # Find our lane pixels first
        leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

        # Fit a second order polynomial to each using `np.polyfit`
        left_fit = np.polyfit(lefty, leftx, 2)
        right_fit = np.polyfit(righty, rightx, 2)

        # Generate x and y values for plotting
        ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
        try:
            left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
            right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
        except TypeError:
            # Avoids an error if `left` and `right_fit` are still none or incorrect
            print('The function failed to fit a line!')
            left_fitx = 1*ploty**2 + 1*ploty
            right_fitx = 1*ploty**2 + 1*ploty

        ## Visualization ##
        # Colors in the left and right lane regions
        out_img[lefty, leftx] = [255, 0, 0]
        out_img[righty, rightx] = [0, 0, 255]

        # Plots the left and right polynomials on the lane lines
        #plt.plot(left_fitx, ploty, color='yellow')
        #plt.plot(right_fitx, ploty, color='yellow')

        return out_img, left_fit, right_fit


    def fit_poly(img_shape, leftx, lefty, rightx, righty):
         ### TO-DO: Fit a second order polynomial to each with np.polyfit() ###
        left_fit = np.polyfit(lefty, leftx, 2)
        right_fit = np.polyfit(righty, rightx, 2)
        # Generate x and y values for plotting
        ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
        ### TO-DO: Calc both polynomials using ploty, left_fit and right_fit ###
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
       
        
        global last_good_left_fitx
        global last_good_right_fitx
        
        lane_width_min = 500
            
        if min(right_fitx-left_fitx)>lane_width_min:  
            last_good_left_fitx=left_fitx
            last_good_right_fitx=right_fitx
            return left_fitx, right_fitx, ploty
        
        else:
            return last_good_left_fitx, last_good_right_fitx, ploty


    def search_around_poly(binary_warped):
        # HYPERPARAMETER
        # Choose the width of the margin around the previous polynomial to search
        # The quiz grader expects 100 here, but feel free to tune on your own!
        margin = 50

        # Grab activated pixels
        nonzero = binary_warped.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])

        ### TO-DO: Set the area of search based on activated x-values ###
        ### within the +/- margin of our polynomial function ###
        ### Hint: consider the window areas for the similarly named variables ###
        ### in the previous quiz, but change the windows to our new search area ###
        left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
                        left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
                        left_fit[1]*nonzeroy + left_fit[2] + margin)))
        right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
                        right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
                        right_fit[1]*nonzeroy + right_fit[2] + margin)))

        # Again, extract left and right line pixel positions
        leftx = nonzerox[left_lane_inds]
        lefty = nonzeroy[left_lane_inds] 
        rightx = nonzerox[right_lane_inds]
        righty = nonzeroy[right_lane_inds]

        # Fit new polynomials
        left_fitx, right_fitx, ploty = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)

        ## Visualization ##
        # Create an image to draw on and an image to show the selection window
        out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
        window_img = np.zeros_like(out_img)
        # Color in left and right line pixels
        out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
        out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

        # Generate a polygon to illustrate the search window area
        # And recast the x and y points into usable format for cv2.fillPoly()
        left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
        left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                                  ploty])))])
        left_line_pts = np.hstack((left_line_window1, left_line_window2))
        right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
        right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                                  ploty])))])
        right_line_pts = np.hstack((right_line_window1, right_line_window2))

        # Draw the lane onto the warped blank image
        cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
        cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
        search_around_prior_result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)

        # Plot the polynomial lines onto the image
        #plt.plot(left_fitx, ploty, color='yellow')
        #plt.plot(right_fitx, ploty, color='yellow')
        ## End visualization steps ##

        return search_around_prior_result, ploty, left_fit, right_fit, left_fitx, right_fitx


    def measure_curvature_pixels():
        '''
        Calculates the curvature of polynomial functions in pixels.
        '''
        # Start by generating our fake example data
        # Make sure to feed in your real data instead in your project!
        #ploty, left_fit, right_fit = generate_data()
        ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
        # Define y-value where we want radius of curvature
        # We'll choose the maximum y-value, corresponding to the bottom of the image
        y_eval = np.max(ploty)

        # Calculation of R_curve (radius of curvature)
        left_curverad = ((1 + (2*left_fit[0]*y_eval + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
        right_curverad = ((1 + (2*right_fit[0]*y_eval + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])

        return left_curverad, right_curverad


    def measure_curvature_real():
        '''
        Calculates the curvature of polynomial functions in meters.
        '''
        # Define conversions in x and y from pixels space to meters
        ym_per_pix = 30/720 # meters per pixel in y dimension
        xm_per_pix = 3.7/600 # meters per pixel in x dimension

        # Start by generating our fake example data
        # Make sure to feed in your real data instead in your project!
        ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
        left_fit_cr = np.polyfit(ploty*ym_per_pix, left_fitx*xm_per_pix, 2)
        right_fit_cr = np.polyfit(ploty*ym_per_pix, right_fitx*xm_per_pix, 2)

        # Define y-value where we want radius of curvature
        # We'll choose the maximum y-value, corresponding to the bottom of the image
        y_eval = np.max(ploty)

        # Calculation of R_curve (radius of curvature)
        left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
        right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])

        return left_curverad, right_curverad


    def vehicle_offset(img_shape, left_fitx, right_fitx, xm_per_pix):
        # vertical image central axis = vehicle central axis
        cen_imgx = img_shape[1]//2

        # vehicle position with respect to the lane center
        veh_pos = (left_fitx[-1] + right_fitx[-1])/2

        #vehicle offset
        veh_offsetx = (cen_imgx - veh_pos) * xm_per_pix

        return veh_offsetx


    def draw_lane_area(binary_warped, left_fitx, right_fitx, Minv, img):

        # Create an image to draw the lines on
        warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
        color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
        #plt.imshow(color_warp)
        # Recast the x and y points into usable format for cv2.fillPoly()
        pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
        pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
        pts = np.hstack((pts_left, pts_right))

        # Draw the lane onto the warped blank image
        cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

        # Warp the blank back to original image space using inverse perspective matrix (Minv)
        newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0])) 
        # Combine the result with the original image
        lane_area_result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)

        return lane_area_result


    def display_curvature_and_veh_position(img):
        
        global framenr
        
        cv2.putText(img, 'Frame #{:}'.format(framenr),
                    (60, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)
        cv2.putText(img, 'Radius of Curvatur: {:.2f} m'.format(curverad), 
                    (60, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)
        if veh_offsetx<0:
            cv2.putText(img, 'Vehicle is {:.2f} m left to center '.format(abs(veh_offsetx)), 
                    (60, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)
        elif veh_offsetx == 0:
            cv2.putText(img, 'Vehicle is on lane center '.format(abs(veh_offsetx)), 
                    (60, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)
        else:
            cv2.putText(img, 'Vehicle is {:.2f} m right to center '.format(abs(veh_offsetx)), 
                    (60, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 1)
        framenr += 1
        return img
    
    img = videoframe

    undist = cv2.undistort(img, mtx, dist, None, mtx)
    thresh_min=20
    thresh_max=100
    binary = color_and_gradient(undist,thresh_min, thresh_max, s_thresh_min, s_thresh_max)
    binary_warped, Minv = unwarp(binary)
    out_img, left_fit, right_fit = fit_polynomial(binary_warped)
    img_shape = binary_warped.shape
    search_around_prior_result, ploty, left_fit, right_fit, left_fitx, right_fitx = search_around_poly(binary_warped)
    left_curverad, right_curverad = measure_curvature_pixels()
    left_curverad, right_curverad = measure_curvature_real()
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/600 # meters per pixel in x dimension
    curverad = (left_curverad + right_curverad)/2
    veh_offsetx = vehicle_offset(img_shape, left_fitx, right_fitx, xm_per_pix)
    lane_area_result = draw_lane_area(binary_warped, left_fitx, right_fitx, Minv, img)
    img_all_infos = display_curvature_and_veh_position(lane_area_result)
    
    # picture in picture for bw image
    front_img = out_img.copy()
        
    scale_percent = 25
    width = int(front_img.shape[1] * scale_percent / 100)
    height = int(front_img.shape[0] * scale_percent / 100)
    topx = img_all_infos.shape[1]-width
    topy = 0
    # resize image
    dsize = (width, height)
    thumbnail_img = cv2.resize(front_img, dsize)
    
    # position thumbnail_img in front_img
    pad = 20
    img_all_infos[topy+pad:height+pad,topx-pad:img_all_infos.shape[1]-pad,:] = thumbnail_img[0:height,0:width,:]


    plt.imshow(img_all_infos)
    return img_all_infos



s_thresh_min=100
s_thresh_max=200

# Beginning and duration subclip
start_sec = 24
end_sec = start_sec + 1

white_output = 'test_videos_output/project_video_s_thresh_'+ str(s_thresh_min) + '-' + str(s_thresh_max) +'V3.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("project_video.mp4")#.subclip(start_sec, end_sec)
framenr = 0
last_good_left_fitx = np.array([])
last_good_right_fitx = np.array([])
        
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)


[MoviePy] >>>> Building video test_videos_output/project_video_s_thresh_100-200V3.mp4
[MoviePy] Writing video test_videos_output/project_video_s_thresh_100-200V3.mp4


100%|█████████▉| 1260/1261 [05:18<00:00,  3.98it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/project_video_s_thresh_100-200V3.mp4 

CPU times: user 2min 55s, sys: 16.7 s, total: 3min 11s
Wall time: 5min 21s
